In [1]:
from utils.database import load_db


start


In [ ]:
import pandas as pd

In [ ]:

def aquire_comparison(table_name: str, name: str):
    df = load_db(table_name)

    location = "data/"

    # read form pandas csv
    df_bm25 = pd.read_csv(location + "bm25_" + name)
    print(df_bm25.columns)
    df_dsi = pd.read_csv(location + "dsi_" + name)
    print(df_dsi.columns)
    print(df.columns)



    # these are the collumn name 
    # query,target_doc_id,rank,model_predictions
    # insert both dataframes on target_doc_id in both datagframes
    df_merged = df_bm25.merge(df_dsi, left_on="target_text_id", right_on="target_doc_id", suffixes=('_bm25', '_dsi'), how='inner')

    # insert left on test_id in df_csv and elaborative_description in df, ignore if there are more than one matches
    df_merged = df_merged.merge(df, left_on="target_doc_id", right_on="elaborative_description", how="left")
    # filter rows where rank_bm25 is 1 and rank_dsi is 1

    # Force string type on both merge keys
    df_merged["target_doc_id"] = df_merged["target_doc_id"].astype(str)
    df["elaborative_description"] = df["elaborative_description"].astype(str)
    df_rank1 = df_merged[(df_merged['rank_bm25'] == 1) & (df_merged['rank_dsi'] == -1)]
    print(f"Number of rows where BM25 rank is 1 and DSI rank is -1: {len(df_rank1)}")

    df_rank2 = df_merged[(df_merged['rank_bm25'] == -1) & (df_merged['rank_dsi'] == 1)]
    print(f"Number of rows where BM25 rank is -1 and DSI rank is 1: {len(df_rank2)}") 


    # save to csv
    print(f"Saving {table_name} to csv...")

    df_rank1.to_csv(location + table_name + "_bm25_rank1_dsi_rank-1.csv", index=False)
    df_rank2.to_csv(location + table_name + "_bm25_rank-1_dsi_rank1.csv", index=False)

    print("Done.")


In [ ]:
aquire_comparison("N10k", "10k_no_thread_results.csv")
aquire_comparison("N100k_thread", "100k_thread_results.csv")
aquire_comparison("N10k_thread", "10k_thread_results.csv")
aquire_comparison("N10k_thread_same_mid", "10k_thread_same_mid_results.csv")





In [ ]:
df_bm25 = pd.read_csv(location + "bm25_" + "10k_thread_same_mid_results.csv")
print(df_bm25.head())

In [ ]:
# list of tables to aquire
tables = [
    ("N10k", "10k_no_thread_results.csv"),
    ("N100k_thread", "100k_thread_results.csv"),
    ("N10k_thread", "10k_thread_results.csv"),
    ("N10k_thread_same_mid", "10k_thread_same_mid_results.csv"),
]


In [ ]:
import pandas as pd
import re

# 1. Load Data
file_dsi = "data/N10k_thread_same_mid_bm25_rank-1_dsi_rank1.csv"
file_bm25 = "data/N10k_thread_same_mid_bm25_rank1_dsi_rank-1.csv"

df_1 = pd.read_csv(file_dsi)
df_2 = pd.read_csv(file_bm25)

# 2. Filter NaNs & Select Top 5
# We only care if NaNs exist in the columns we are actually going to print
cols_to_check = [
    'target_text_id', 'query_bm25', 'body', 'body_clean_and_subject',
    'doctoquery', 'text_rank_query', 
    'model_predictions_bm25', 'model_predictions_dsi'
]

# Drop rows with missing values in these specific columns
df_1_clean = df_1.dropna(subset=cols_to_check)
df_2_clean = df_2.dropna(subset=cols_to_check)

# Now take the top 5
df_1_top = df_1_clean.head(5).copy()
df_1_top['Group_Label'] = "DSI Correct / BM25 Incorrect"

df_2_top = df_2_clean.head(5).copy()
df_2_top['Group_Label'] = "BM25 Correct / DSI Incorrect"

df = pd.concat([df_1_top, df_2_top])

# 3. Robust LaTeX Escape Function
def clean_tex(text):
    if pd.isna(text):
        return ""
    text = str(text)
    
    replacements = {
        '&': r'\&', '%': r'\%', '$': r'\$', '#': r'\#', '_': r'\_',
        '{': r'\{', '}': r'\}', '~': r'\textasciitilde{}', '^': r'\^{}',
        '\\': r'\textbackslash{}', '<': r'\textless{}', '>': r'\textgreater{}'
    }
    
    pattern = re.compile('|'.join(re.escape(key) for key in replacements.keys()))
    safe_text = pattern.sub(lambda x: replacements[x.group()], text)
    
    # Remove newlines for standard text fields
    return safe_text.replace('\n', ' ').replace('\r', '')

# 4. Helper for Predictions
def format_preds(val):
    if pd.isna(val):
        return ""
    parts = str(val).split('|')
    cleaned = [clean_tex(p.strip()) for p in parts if p.strip()]
    return r' \newline '.join(cleaned)

# 5. Helper for Body Text
def format_body(val, limit=400):
    val_str = str(val)
    if len(val_str) > limit:
        val_str = val_str[:limit-3] + "..."
    return clean_tex(val_str)

# 6. Generate LaTeX
latex_lines = []

latex_lines.append(r"\begin{longtable}{|p{3.5cm}|p{11.5cm}|}")
latex_lines.append(r"\caption{Comparison: Body vs Cleaned Input vs Predictions} \label{tab:detailed_comparison} \\")
latex_lines.append(r"\hline")
latex_lines.append(r"\textbf{Attribute} & \textbf{Content} \\")
latex_lines.append(r"\hline")
latex_lines.append(r"\endfirsthead")
latex_lines.append(r"\hline")
latex_lines.append(r"\textbf{Attribute} & \textbf{Content} \\")
latex_lines.append(r"\hline")
latex_lines.append(r"\endhead")

current_group = None

for idx, row in df.iterrows():
    
    if row['Group_Label'] != current_group:
        current_group = row['Group_Label']
        latex_lines.append(r"\multicolumn{2}{|c|}{\textbf{\textit{" + current_group + r"}}} \\")
        latex_lines.append(r"\hline")

    target_id = clean_tex(row['target_text_id'])
    query = clean_tex(row['query_bm25'])
    
    body_raw = format_body(row['body'], limit=500)
    body_clean = format_body(row['body_clean_and_subject'], limit=500)
    
    d2q = clean_tex(row['doctoquery'])
    tr = clean_tex(row['text_rank_query'])
    
    preds_bm25 = format_preds(row['model_predictions_bm25'])
    preds_dsi = format_preds(row['model_predictions_dsi'])
    
    # --- ROW BLOCK ---
    latex_lines.append(f"\\textbf{{Target ID}} & {target_id} \\\\")
    latex_lines.append(r"\hline")
    latex_lines.append(f"\\textbf{{Query}} & {query} \\\\")
    latex_lines.append(r"\hline")
    latex_lines.append(f"\\textbf{{Body (Raw)}} & {body_raw} \\\\")
    latex_lines.append(r"\hline")
    latex_lines.append(f"\\textbf{{Body (Clean+Subj)}} & {body_clean} \\\\")
    latex_lines.append(r"\hline")
    latex_lines.append(f"\\textbf{{Doc2Query}} & {d2q} \\\\")
    latex_lines.append(r"\hline")
    latex_lines.append(f"\\textbf{{TextRank}} & {tr} \\\\")
    latex_lines.append(r"\hline")
    latex_lines.append(f"\\textbf{{Preds (BM25)}} & {preds_bm25} \\\\")
    latex_lines.append(r"\hline")
    latex_lines.append(f"\\textbf{{Preds (DSI)}} & {preds_dsi} \\\\")
    
    latex_lines.append(r"\hline \hline") 

latex_lines.append(r"\end{longtable}")

with open('tbls.tex', 'w', encoding='utf-8') as f:
    f.write('\n'.join(latex_lines))

print(f"Successfully generated 'tbls.tex'. Filtered {len(df_1)-len(df_1_clean)} rows from DSI and {len(df_2)-len(df_2_clean)} rows from BM25 due to missing values.")

In [ ]:
df_1.head()